## What is LightGBM Modling in ML?

1. based on ensemble, GBM
    - ensemble
        - 독립적인 개체들간의 다수결 투표를 통해서 대표 모델 선출(bagging)
        - 특정 모델의 output을 다른 모델의 input에 사용하는 방법으로 과중치를 부여하는 방법(boosting)
    - GBM (Gradient Boosting Machine)
        - Boosting Model에서 사용하는 가중치를 Gradient로 사용한 기법
        - 데이터 정규화나 아웃라이어의 별도 처리가 없이 범용적으로 사용할 수 있다. 대표적인 모델 : XGBoost
2. 단점 극복을 위한 방법론 채택
    - GOSS (Gradient-based One-Side Sampling) : low gradient 데이터를 배제함으로써 size를 줄임
    - EFB (Exclusive Feature Bundling) : 서로 배타적인 feature들만 골라 사용함으로써, dimension을 줄임
3. 다른 Boosting 알고리즘과 차별화되는 특징
    - Categorial Feature의 자동 변환 지원
    - 메모리 사용량이 적음
    - leaf-wise 트리 분할 방식을 사용 _ 대부분의 트리 기반 알고리즘은 level-wise 트리 분할 방식을 사용
    - Data Set 크기가 클 때 뛰어난 성능
    - 10,000건 이하의 Data set에서는 과적합이 잘 발생
4. 기타 특징
    - When adding a new tree node, LightGBM chooses the split point that has the largest gain.
5. 정리
    - 장점
        - 학습하는데 시간이 짧다. (통상 XGBoost 학습속도의 1.3배 ~ 1.5배)
        - 메모리 사용량이 상대적으로 적다.
        - 대용량 데이터 처리 가능
    - 적은 데이터셋에서는 오버피팅 발생 가능 (10,000 이하)

### about main parameter

- num_leaves
    - 하나의 트리가 가질 수 있는 최대 리프 개수
    - default : 31
    - 클수록 정확도는 높아지지만, 오버피팅 발생 가능
    - main parameter to control the complexity of the tree model
    - max num : 2^(max_depth)
        - For example, when the <ins>max_depth=$7$</ins> the depth-wise tree can get good accuracy, but setting num_leaves to <ins>$127$</ins> may cause over-fitting, and setting it to <ins>$70 or 80$</ins> may get better accuracy than depth-wise.
    - Decrease num_leaves to reduce training time<br>
<br>
- min_data_in_leaf
    - 한 리프의 최소 데이터 수 (decision tree의 min_sample_leaf와 동일, 오버피팅 제어)
    - default : 20
    - 클수록 오버피팅 방지
    - Minimum number of observations that must fall into a tree node for it to be added.
    - This is a very important parameter to prevent over-fitting in a leaf-wise tree
    - Its optimal value depends on the number of training samples and num_leaves
    - In practice, setting it to hundreds or thousands is enough for a large dataset.<br>
<br>
- max_depth
    - 트리의 깊이
    - default : 1
    - 위 두개 파라미터와 결합하여 오버피팅 방지
    - -1로 설정하면 제한없이 분기한다.
    - feature가 많다면 크게 설정
    - 파라미터 설정 시 우선적으로 설정
    - controls the maximum distance
    - You also can use max_depth to limit the tree depth explicitly

### about sub parameter

- min_sum_hessian_in_leaf
    - Minimum sum of the Hessian (second derivative of the objective function evaluated for each observation) for observations in a leaf.<br><br>
- num_iterations
    - 반복 수행하려는 트리의 개수 (너무 크면 오버피팅 발생)
    - default : 100
    - 일반적으로 100 ~ 1000 값이 좋다.
    - controls the number of boosting rounds that will be performed
    - If you try changing num_iterations, change the <ins>learning_rate</ins> as well
    - Decrease num_iterations to reduce training time.<br><br>
- early_stopping_round
    - If early stopping is enabled, after each boosting round the model’s training accuracy is evaluated against a validation set that contains data not available to the training process
    - For example, early_stopping_round=1 says “the first time accuracy on the validation set does not improve, stop training”.<br><br>
- learning_rate
    - 부스팅 스탭 반복할 때 학습률. 0~1 사이의 값
    - 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다.
    - 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
    - default : 0.1<br><br>
- objective
    - 수치예측이면 regression, 이진분류이면 binary
    - <ins>사용하는 데이터셋의 타겟팅 값의 형태에 따라 조정 필요</ins>
    - default : regression<br><br>
- boosting
    - 부스팅 방법 (gbdt: Gradient Boosting DecisionTree / rf: RandomForest)
    - default : gbdt
        - gbdt : traditional Gradient Boosting Cecision Tree, aliases : gbrt
        - rf : Random Forest, aliases : random_forest
        - dart : Dropouts meet Multiple Additive Regression Trees
        - goss : Gradient - bsed One-side Sampling<br><br>
- bagging_fraction
    - 데이터 샘플링 비율, 오버피팅 제어
    - 범위 : 0 < fraction <= 1, 0이 되지 않도록 해야함
    - default : 1.0
    - 배깅을 하기 위해서 데이터를 랜덤 샘플링하여 학습에 사용한다.<br><br>
- feature_fraction
    - 개별 트리 학습 시 무작위로 선택하는 feature의 비율
    - 1보다 작다면 LGBM은 매 iteration(tree)마다 다른 featrue를 랜덤하게 추출하여 학습하게 된다.
    - 만약 0.8로 값을 설정하면 매 tree를 구성할 때, feature의 80%만 랜덤하게 선택한다.
    - 과적합을 방지하기 위해 사용할 수 있으며, 학습 속도가 향상한다.
    - default : 1.0 <br><br>
- lambda_l1, lambda_l2
    - L1 regulation 제어, L2 regulation 제어
    - 0.0<br><br>
- metric
    - 성능평가를 어떤 것으로 할 것인지(auc, l1, l2 등)
    - <ins>성능평가를 어떤 것으로 할 것인지 조정 필요</ins>
    - " "

### 참고링크
- <href>https://blog.naver.com/ilovelatale/222298514382</href> : Kaggle case study - LightGBM
- <href>https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html</href> : LightGBM - Parameters Tuning
- <href>https://for-my-wealthy-life.tistory.com/24</href> : lightGBM이란? 파라미터 설명과 코드 실습
- <href>https://greatjoy.tistory.com/72</href> : [LightGBM] LGBM는 어떻게 사용할까? (설치,파라미터튜닝)

## defalut setting

## import

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

### Memory reduce

In [4]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

### read dataFrame

In [9]:
train = pd.read_csv('/Users/sanghyun/Desktop/Coding/PUBG_ML_Project/data/featured_train_4.csv')
train = reduce_mem_usage(train)

In [10]:
## 원본 데이터 보존을 위해 train data copy

data = train.copy()

## Modeling

### Data Split

In [11]:
X = data.drop(columns='winPlacePerc')
y = data['winPlacePerc']

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=100)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1621395, 46), (405349, 46), (1621395,), (405349,))

### LigthGBM

#### default Modeling

In [12]:
# LGBMRegressor

model_lgb = LGBMRegressor().fit(X_train, y_train)
pred4 = model_lgb.predict(X_test)
mae = mean_absolute_error(pred4, y_test)
print('LGBMRegressor MAE : %f' %mae)

LGBMRegressor MAE : 0.046707


#### HyperparameterTuning 1